In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from utils import load_config, set_seed
from models import MAIRA2Model
from attribution import (
    CoIBAForLVLM,
    LayerAttribution,
    TokenAttribution,
    visualize_attribution,
    plot_attribution_grid,
)

In [ ]:
set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

## 1. Load Model and Configuration

In [ ]:
# Load config
config = load_config(project_root / "configs" / "coiba_config.yaml")

# Load model (uncomment when ready)
# model = MAIRA2Model.from_pretrained(
#     checkpoint=config["model"]["checkpoint"],
#     device=device,
#     load_in_8bit=True,
# )

## 2. Generate Attributions with CoIBA

In [ ]:
def generate_coiba_attribution(model, image, input_ids, target_token_idx):
    """Generate CoIBA attribution for a target token."""
    coiba = CoIBAForLVLM(model, config.get("coiba", {}))
    
    attribution = coiba.generate_attribution(
        image=image,
        input_ids=input_ids,
        target_token_idx=target_token_idx,
        method="integrated_gradients",
        n_steps=50,
    )
    
    return attribution

# Example (uncomment when model loaded):
# image_tensor = ...  # Load and preprocess image
# input_ids = ...     # Tokenise prompt
# attr = generate_coiba_attribution(model, image_tensor, input_ids, target_token_idx=100)

## 3. Visualise Attribution Maps

In [ ]:
# Create synthetic example for demonstration
dummy_image = np.random.rand(224, 224)
dummy_attribution = np.random.rand(224, 224)

# Centre the attribution to simulate focused region
y, x = np.ogrid[:224, :224]
centre_y, centre_x = 112, 112
mask = np.exp(-((x - centre_x)**2 + (y - centre_y)**2) / (2 * 40**2))
dummy_attribution = dummy_attribution * mask

fig = visualize_attribution(
    image=dummy_image,
    attribution_map=dummy_attribution,
    title="Sample Attribution Map",
    cmap="jet",
    alpha=0.5,
    show=True,
)

## 4. Compare Attribution Methods

In [ ]:
def compare_methods(model, image, input_ids, target_token_idx):
    """Compare different attribution methods."""
    methods = ["integrated_gradients", "noise_tunnel"]
    results = {}
    
    coiba = CoIBAForLVLM(model)
    
    for method in methods:
        attr = coiba.generate_attribution(
            image=image,
            input_ids=input_ids,
            target_token_idx=target_token_idx,
            method=method,
            n_steps=20,
        )
        results[method] = attr
    
    return results

# Visualise comparison (placeholder)
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(dummy_image, cmap="gray")
axes[0].set_title("Original Image")
axes[1].imshow(dummy_attribution, cmap="jet")
axes[1].set_title("Integrated Gradients")
axes[2].imshow(dummy_attribution * 0.8 + np.random.rand(224, 224) * 0.2, cmap="jet")
axes[2].set_title("SmoothGrad")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

## 5. Token-Level Attribution Analysis

In [ ]:
def plot_token_importance(tokens, importance_scores):
    """Plot token importance scores."""
    fig, ax = plt.subplots(figsize=(12, 4))
    
    colours = plt.cm.RdYlGn(importance_scores / importance_scores.max())
    
    ax.bar(range(len(tokens)), importance_scores, color=colours)
    ax.set_xticks(range(len(tokens)))
    ax.set_xticklabels(tokens, rotation=45, ha="right")
    ax.set_ylabel("Importance Score")
    ax.set_title("Token-Level Attribution")
    
    plt.tight_layout()
    plt.show()

# Example with dummy data
tokens = ["The", "heart", "is", "enlarged", "with", "cardiomegaly"]
scores = np.array([0.1, 0.8, 0.05, 0.9, 0.1, 0.95])
plot_token_importance(tokens, scores)

## 6. Next Steps

- Apply to real MAIRA-2 outputs
- Compare with ground truth anatomical regions
- Quantify attribution quality metrics